In [1]:
import torch
from torch import Tensor
import torch.nn as nn
import torch.nn.functional as F
from typing import Any, Callable, List, Optional, Tuple

CNN                 INCEPTION             RESNET ![Alt text](../images/Screenshot%20from%202023-03-24%2014-24-10.png)

Identity Block ![Alt text](../images/idblock.png)

Reduce Block ![Alt text](../images/reduceblock.png)

In [2]:
class BasicConv2d(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, stride=1, padding=0, dilation=1, groups=1, bias=False):
        super(BasicConv2d, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=kernel_size,stride=stride,padding=padding,bias=bias)
        self.norm = nn.BatchNorm2d(out_channels, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)

    def forward(self,x):
        x = self.conv1(x)
        x = self.norm(x)
        return x


In [3]:
class BottleNeck(nn.Module):
    def __init__(self, prev_channels, in_channels, out_channels, kernel_size=3, stride=2, padding=1, reduce=False):
        super(BottleNeck, self).__init__()
        self.reduce = reduce
        
        self.ReduceBlock1 = BasicConv2d(prev_channels, in_channels, kernel_size=1, stride=stride, padding=0)
        self.ReduceBlock2 = BasicConv2d(prev_channels, out_channels, kernel_size=1, stride=stride, padding=0)

        self.Block1 = BasicConv2d(prev_channels, in_channels, kernel_size=1, stride=1, padding=0)
        self.Block2 = BasicConv2d(in_channels, in_channels, kernel_size=kernel_size, stride=1, padding=padding)
        self.Block3 = BasicConv2d(in_channels, out_channels, kernel_size=1, stride=1, padding=0)
        self.relu = nn.ReLU()
        
    def forward(self, x):
        out = x
        if self.reduce:
            out = self.ReduceBlock1(x)
            out = self.relu(out)
            identity = self.ReduceBlock2(x)
        else:
            out = self.Block1(out)
            out = self.relu(out)
        out = self.Block2(out)
        out = self.relu(out)
        out = self.Block3(out)
        if self.reduce:
            out = self.relu(out+identity)
        return out
        

ResNet50 implementation with the following architecture: ![Alt text](../images/resnet_architectures.png)

In [4]:
class ResNet(nn.Module):
    def __init__(self, num_classes):
        super(ResNet, self).__init__()

        #Stage 1
        self.conv1 = BasicConv2d(3, 64, 7, 2, 3)
        self.pool1 = nn.MaxPool2d(kernel_size=3,stride=2)

        #Stage 2
        self.ResBlock2a = BottleNeck(64, 64, 256, 3, 1, 1, reduce=True)
        self.ResBlock2b = BottleNeck(256, 64, 256, 3)
        self.ResBlock2c = BottleNeck(256, 64, 256, 3)

        #Stage 3
        self.ResBlock3a = BottleNeck(256, 128, 512, 3, 2, 1, reduce=True)
        self.ResBlock3b = BottleNeck(512, 128, 512, 3)
        self.ResBlock3c = BottleNeck(512, 128, 512, 3)
        self.ResBlock3d = BottleNeck(512, 128, 512, 3)

        #Stage 4
        self.ResBlock4a = BottleNeck(512, 256, 1024, 3, 2, 1, reduce=True)
        self.ResBlock4b = BottleNeck(1024, 256, 1024, 3)
        self.ResBlock4c = BottleNeck(1024, 256, 1024, 3)
        self.ResBlock4d = BottleNeck(1024, 256, 1024, 3)
        self.ResBlock4e = BottleNeck(1024, 256, 1024, 3)
        self.ResBlock4f = BottleNeck(1024, 256, 1024, 3)

        #Stage 5
        self.ResBlock5a = BottleNeck(1024, 512, 2048, 3, 2, 1, reduce=True)
        self.ResBlock5b = BottleNeck(2048, 512, 2048, 3)
        self.ResBlock5c = BottleNeck(2048, 512, 2048, 3)

        self.avgpool = nn.AdaptiveAvgPool2d((1,1))
        self.fc = nn.Linear(2048, num_classes)
        self.classifier = nn.Softmax(dim=1)

    def forward(self, x):
        x = self.conv1(x)
        x = self.pool1(x)
        x = self.ResBlock2a(x)
        x = self.ResBlock2b(x)
        x = self.ResBlock2c(x)
        x = self.ResBlock3a(x)
        x = self.ResBlock3b(x)
        x = self.ResBlock3c(x)
        x = self.ResBlock3d(x)
        x = self.ResBlock4a(x)
        x = self.ResBlock4b(x)
        x = self.ResBlock4c(x)
        x = self.ResBlock4d(x)
        x = self.ResBlock4e(x)
        x = self.ResBlock4f(x)
        x = self.ResBlock5a(x)
        x = self.ResBlock5b(x)
        x = self.ResBlock5c(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x



In [5]:
resnet = ResNet(1000)
x = torch.randn(1,3,224,224)
output = resnet(x)
print(output.shape)

torch.Size([1, 2048])
